In [16]:
!conda install -c apple tensorflow-deps==2.7

Solving environment: done

# All requested packages already installed.



In [17]:
!conda list | grep tensor

tensorboard               2.6.0              pyhd8ed1ab_1    conda-forge
tensorboard-data-server   0.6.0            py38h10d4710_1    conda-forge
tensorboard-plugin-wit    1.8.1              pyhd8ed1ab_0    conda-forge
tensorflow-deps           2.7.0                         0    apple
tensorflow-estimator      2.7.0                    pypi_0    pypi
tensorflow-macos          2.7.0                    pypi_0    pypi
tensorflow-metal          0.3.0                    pypi_0    pypi


In [4]:
!python -m pip install tensorflow-macos==2.8


     |████████████████████████████████| 179.0 MB 60.4 MB/s            
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached libclang-13.0.0-py2.py3-none-macosx_11_0_arm64.whl (12.2 MB)
  Using cached tensorflow_estimator-2.7.0-py2.py3-none-any.whl (463 kB)


In [5]:
!python -m pip install tensorflow-metal


     |████████████████████████████████| 1.2 MB 26.7 MB/s            


In [25]:
!python -m pip install tensorflow_datasets

  Using cached tensorflow_datasets-4.5.0-py3-none-any.whl (4.2 MB)
     |████████████████████████████████| 76 kB 16.5 MB/s            
  Using cached promise-2.3.tar.gz (19 kB)
  Preparing metadata (setup.py) ... done
  Using cached tensorflow_metadata-1.6.0-py3-none-any.whl (48 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached googleapis_common_protos-1.54.0-py2.py3-none-any.whl (207 kB)
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21503 sha256=65decf15a9e89f326c50061f610ab097f0847a064923739d133a1e380cf0bb09
  Stored in directory: /Users/shawn/Library/Caches/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise


In [28]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

import time

In [21]:
sys.version

'3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:21:17) \n[Clang 11.1.0 ]'

In [22]:
tf.__version__

'2.7.0'

In [13]:
from tensorflow.keras import layers
from tensorflow.keras import models
import time


Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-01-27 15:36:11.269913: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-27 15:36:11.270111: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [29]:
import tensorflow_datasets as tfds

tf.enable_v2_behavior()

from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [33]:
start = time.time()
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

batch_size = 128

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

print("time :", time.time()-start) # 1분 28.6초 / 1분 43.6초

2022-01-27 16:02:04.920230: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 16:02:04.961207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 16:02:05.037623: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 16:02:05.079802: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 16:02:05.139000: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 16:02:05.184906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 16:02:05.404631: I tensorflow/core/grappler/optimizers/cust

Train on 469 steps, validate on 79 steps
Epoch 1/12


2022-01-27 16:02:05.619610: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


467/469 [============================>.] - ETA: 0s - batch: 233.0000 - size: 1.0000 - loss: 0.1589 - accuracy: 0.9536

/Users/shawn/miniforge3/envs/py38-tf/lib/python3.8/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2022-01-27 16:02:13.463883: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 9s 15ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.1586 - accuracy: 0.9537 - val_loss: 0.0519 - val_accuracy: 0.9843
Epoch 2/12
469/469 [==============================] - 9s 15ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0442 - accuracy: 0.9863 - val_loss: 0.0434 - val_accuracy: 0.9862
Epoch 3/12
469/469 [==============================] - 8s 15ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0282 - accuracy: 0.9909 - val_loss: 0.0345 - val_accuracy: 0.9878
Epoch 4/12
469/469 [==============================] - 8s 15ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0192 - accuracy: 0.9941 - val_loss: 0.0378 - val_accuracy: 0.9885
Epoch 5/12
469/469 [==============================] - 8s 15ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0148 - accuracy: 0.9954 - val_loss: 0.0326 - val_accuracy: 0.9898
Epoch 6/12
469/469 [==============================] - 8s 15ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0109 - accuracy: 0.99

In [32]:
start = time.time()
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

batch_size = 128

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

print("time :", time.time()-start) # 1분 20.9초 / 1분 21.4초 / 1분 23.1초

2022-01-27 15:59:54.193740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 15:59:54.221540: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 15:59:54.278450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 15:59:54.301389: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 15:59:54.346754: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 15:59:54.378221: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-27 15:59:54.455023: I tensorflow/core/grappler/optimizers/cust

Train on 469 steps, validate on 79 steps
Epoch 1/12
469/469 [==============================] - ETA: 0s - batch: 234.0000 - size: 1.0000 - loss: 0.1495 - accuracy: 0.9542

/Users/shawn/miniforge3/envs/py38-tf/lib/python3.8/site-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2022-01-27 16:00:00.959106: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 7s 12ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.1495 - accuracy: 0.9542 - val_loss: 0.0486 - val_accuracy: 0.9847
Epoch 2/12
469/469 [==============================] - 7s 12ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0438 - accuracy: 0.9867 - val_loss: 0.0363 - val_accuracy: 0.9878
Epoch 3/12
469/469 [==============================] - 7s 12ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0276 - accuracy: 0.9914 - val_loss: 0.0363 - val_accuracy: 0.9884
Epoch 4/12
469/469 [==============================] - 7s 12ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0182 - accuracy: 0.9944 - val_loss: 0.0347 - val_accuracy: 0.9897
Epoch 5/12
469/469 [==============================] - 7s 13ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0134 - accuracy: 0.9959 - val_loss: 0.0361 - val_accuracy: 0.9895
Epoch 6/12
469/469 [==============================] - 7s 12ms/step - batch: 234.0000 - size: 1.0000 - loss: 0.0085 - accuracy: 0.99